# メモ書き
## MIXED PRECISION TRAINING
### 1 INTRODUCTION
- 最近のDNNモデルは徐々に巨大化しており、それに伴いより多くの計算やメモリーへのリソースが学習の際に必要となる。それらの必要となるリソースは、精度を低くすることにより、減らすことができる。
- 学習や推論等を行う速度は以下の3つによって決まる。
    1. 演算速度
    2. メモリ速度
    3. レイテンシ<br>
    最初の2つは扱う精度を低くすることにより、向上させることができる。<br>
        - 精度を低くすることにより(例:FP32→FP16)、同じ値をより少ないビット数でメモリに持たせることができる。
        - より少ない精度を使用することにより、演算しなければならないビット数も減らせることができる。例を挙げると、FP16を使用して最新のGPUで演算を行うと、FP32で演算した時と比べて、2倍～8倍のスループットを実現する。
- 今回提案する手法は、FP32の代わりにFP16を使用し、様々なNNモデルでaccuracyを保ちつつ、演算速度をあげることを検証する。
- FP32からFP16に代えても、モデルのaccuracyを保つために以下3つの提案する。
    1. FP32で重みの値を保持する
    2. 勾配が0になることを防ぐためのロススケーリング
    3. FP16とFP32の混合演算
    <br><span style="color: red; ">**※**</span>なお上3つのことを行う上で、モデルやハイパーパラメータを変更する必要はない
    
### 2 RELATED WORK
- 当論文で提案する手法は以下3つの点で今までの手法とは異なる。
    1. 全ての順伝播と逆伝播でFP16を用いて演算する
    2. FP16を使用する際もハイパーパラメータを変更する必要がない(レイヤーのノード数を変更する等)
    3. 当手法でFP16を用いて学習させたモデルは、FP32で学習させたモデルと比べて、同精度である

### 3 IMPLEMENTATION
#### 3.1 FP32での重みの保持
- FP32で重みを保持することが(精度を保つ上で)当たり前というわけではないが、以下2つの理由で多くのネットワークで必要する。
    1. 重みを更新する際に、更新する値、つまり重みの勾配 x 学習率、がFP16では0になり得る。そのため、重みの更新の際はFP32を使用することでこの問題を解決できる。
    2. 重みの更新値に対して重みの値がとても大きいとき。**これについては詳細を調査する必要あり**
- 重みをFP32で保持することが必要であることを示すために、Mandarin(中国の標準語)スピーチモデルを用いて検証したところ、混合演算を行った結果はFP32で演算した結果と同程度の精度を出すことができたが、FP16で重みの更新をしたところ、FP32で演算した結果の80%も精度を失った。
- 混合演算の際に追加で重みをFP32で保持することは、FP32で演算するときと比べて50%ほどメモリーの容量を多くとるが、総合的に言うと、圧倒的にメモリー使用量を削減できる。その理由は、学習時のメモリー使用量の大部分の原因はアクティベーションのところであり(より大きなバッチサイズを使用したり、逆伝播の演算の際にアクティベーションの値を使用するため、保持する必要がある)、その全てのアクティベーションがFP16で演算されるため、DNNを学習させるのに必要なメモリー量を大まかに言って約半分ほどに削減できる。
#### 3.2 ロススケーリング
- 下図はFP32を用いてSSDを学習させたときに、全てのレイヤーから得た活性化の勾配値のヒストグラムである。
- 下図から見てわかるように、FP16で扱うことができる範囲にほとんど値がなく、多くの値はFP16で扱うことができる最小値より小さい値である(その場合FP16では扱うことができないため、FP16を用いて演算を行った場合0になる)。
- そのため、大部分の活性化の勾配値をカバーするためにはスケールアップすることが必要であり(以下のケースでは8倍する)、そうすることによりFP32を用いた場合と同精度を出すことができる。
![loss_scaling](https://user-images.githubusercontent.com/37681936/71586989-64211a80-2b5f-11ea-8fb3-70b25a2bb496.png)

***
## Training With Mixed Precision :: DEEP LEARNING SDK DOCUMENTATION
- Mixed Presicion Trainingを使用するためには、以下2つのステップが必要。<br>
    1. 計算上FP16で事が足りる箇所はFP16を適用する<br>
    2. 勾配の値が小さくなっても保持するためにロススケーリングを使用する<br>
- より小さな精度で演算を行うメリットは以下の2つ<br>
    1. **メモリー使用量を削減**<br>
    例を挙げると、普段使用する精度はFP32(32ビット必要)。対してFP16(16ビット必要)。すなわちFP32からFP16に変更することで16ビッド(32ビット-16ビット)も削減できる。そうすることにより、より大きなモデル、またはより大きなミニバッチで学習させることができる。<br>
    2. **学習時間、または推論時間の短縮**<br>
    上の例で挙げたように、FP16を使用することにより、FP32を使用する時と比べて、アクセスするバイト数を半分(16ビット)にできる。そのため、計算時間も短縮できる。NVIDIAのGPUでは演算にFP16を使用すると、FP32で計算する時と比べて、最大8倍のスループットを実現する。